In [1]:
def readJson(f):
    for l in open(f):
        yield eval(l)

In [3]:
from collections import defaultdict
user = {}
item = {}
userdict = {}
itemdict = {}
for l in readJson('amazon_reviews_Electronics/train.json'):
    u,i = l['reviewerID'],l['asin']
    helpful,outof = l['helpful']['nHelpful'],l['helpful']['outOf']
    summary, rating = l['summary'], l['overall']
    if u not in userdict:
        userdict[u] = {'bias':0}
    if u not in user:
        user[u] = defaultdict(list)
    userdict[u][i] = (helpful/outof, outof)
    user[u]['helpful'].append(helpful/outof)
    user[u]['rating'].append(rating)
    user[u]['outof'].append(outof)
    user[u]['summary'].append(summary)
    if i not in itemdict:
        itemdict[i] = {'bias':0}
    if i not in item:
        item[i] = defaultdict(list)
    itemdict[i][u] = (helpful/outof, outof)
    item[i]['helpful'].append(helpful/outof)
    item[i]['rating'].append(rating)
    item[i]['outof'].append(outof)
    item[i]['summary'].append(summary)

In [4]:
user1 = {}
for u in user:
    user1[u] = {}
    user1[u]['outof'] = sum(user[u]['outof'])/len(user[u]['outof'])
    user1[u]['helpful'] = sum(user[u]['helpful'])/len(user[u]['helpful'])
    user1[u]['summary'] = user[u]['summary'][0]
    user1[u]['rating'] = user[u]['rating'][0]

In [5]:
import pandas as pd
user_df = pd.DataFrame.from_dict(user1,orient='index')
user_df.head()

,helpful,summary,rating,outof
000036e73a711d591b1698d886592ddd47ccca6468915e2ff35cf6b57c94dd22,1.000000,Great Sound - Poor Digital Connectivity,3.0,4.0
00003ebc086607421779c0a404a99066d8b7f59b78fb42681aa0a93ff00b3b9c,1.000000,purple macpro cover,4.0,1.0
00004a3fc7b4b7e3b4e4051688af54f88715c80c899be020c00ef59cac7f6017,0.750000,Garbage,1.0,4.0
00005a3402fbf236e2f1aab4c6be7148a5b1ec19629c4e898707b749e29512f0,0.666667,Surprisingly mediocre sound from Sennheiser,2.0,21.0
0000762647ebbd67dfa8da4cd2eb37304f70e1e57fbd9dd0c922dfc6ce425998,1.000000,Iomega super slim dvd,5.0,8.0


In [6]:
item1 = {}
for i in item:
    item1[i] = {}
    item1[i]['outof'] = sum(item[i]['outof'])/len(item[i]['outof'])
    item1[i]['helpful'] = sum(item[i]['helpful'])/len(item[i]['helpful'])
    item1[i]['summary'] = item[i]['summary'][0]
    item1[i]['rating'] = item[i]['rating'][0]
item_df = pd.DataFrame.from_dict(item1,orient='index')
item_df.head()

,helpful,summary,rating,outof
000036af4a51385524ab9a0c0c290a212529efdfab8e2ee0ec54b3b9b8a8a5c1,0.937500,Istanbul Turkey Support + Bad DVD/CD Burner NO...,1.0,7.250000
00007d5c5e8f16abdd5f8cdc022a34aedbdb6d29b52fd60ae8e4459847d39a0d,0.000000,Drive Fits,5.0,1.000000
00009ad21e63ebd13105074ee0da620d382e114b2cb8a0e8e8f209da45fd1a3c,0.683333,Decent Buy!,4.0,1.400000
0000bfd79a739d0b85dfb7867e75cf9f5b1ed2cfb77b88c8f21db3cad7feb8fb,1.000000,Best MP3 Case Ever!!!,5.0,2.000000
0000f141af42b9b562edc1f3e07aeaeb3b8576d56f93963f7019e1afcf249102,0.829630,Inconsistent Pictures,2.0,3.222222


In [58]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer1 = CountVectorizer(stop_words='english')
vectorizer1.fit(user_df.summary)
sum_vec1 = vectorizer1.transform(user_df.summary)
vectorizer2 = CountVectorizer(stop_words='english')
vectorizer2.fit(item_df.summary)
sum_vec2 = vectorizer2.transform(item_df.summary)

In [59]:
from sklearn.decomposition import TruncatedSVD
svd1 = TruncatedSVD(n_components=100, algorithm='arpack')
sum_lsa1 = svd1.fit_transform(sum_vec1)
svd2 = TruncatedSVD(n_components=100, algorithm='arpack')
sum_lsa2 = svd2.fit_transform(sum_vec2)

In [60]:
from sklearn.preprocessing import normalize
sum_lsa1 = normalize(sum_lsa1)
sum_df1 = pd.DataFrame(sum_lsa1, columns=list(range(100)),index=user_df.index)
# print (sum_df1.head())
sum_lsa2 = normalize(sum_lsa2)
sum_df2 = pd.DataFrame(sum_lsa2, columns=list(range(100)),index=item_df.index)
# print (sum_df2.head())

In [61]:
user_df1 = pd.concat([user_df.drop('summary', axis=1), sum_df1], axis=1)
# print (user_df1.head())
item_df2 = pd.concat([item_df.drop('summary', axis=1), sum_df2], axis=1)
# print (item_df2.head())

In [62]:
from sklearn.cluster import MiniBatchKMeans
kmeans1 = MiniBatchKMeans(5000, batch_size=5000, random_state=0)
kmeans1.fit(user_df1.drop('helpful', axis=1))
kmeans2 = MiniBatchKMeans(2000, batch_size=2000, random_state=0)
kmeans2.fit(item_df2.drop('helpful', axis=1))

MiniBatchKMeans(batch_size=2000, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=2000, n_init=3, random_state=0, reassignment_ratio=0.01,
        tol=0.0, verbose=0)

In [18]:
# compute global mean
sum_score = count = 0
sum_votes = 0
for item in itemdict:
    for user in itemdict[item]:
        if user != 'bias':
            count += 1
            sum_score += itemdict[item][user][0]
            sum_votes += itemdict[item][user][1]
            
mu = sum_score/count
mu_votes = sum_votes/count
mu

0.7443082056168422

In [65]:
# average votes a review gets
mu_votes

7.033746

In [20]:
# compute user and item bias
l = 1 # lambda
while True:
    diff = 0
    for item in itemdict:
        sum_rating = 0
        for user in itemdict[item]:
            if user != 'bias':
                sum_rating += itemdict[item][user][0] - mu - userdict[user]['bias']
        tmp = itemdict[item]['bias']
        itemdict[item]['bias'] = sum_rating/(l + len(itemdict[item]) - 1)
        diff += abs(tmp - itemdict[item]['bias'])
    for user in userdict:
        sum_rating = 0
        for item in userdict[user]:
            if item != 'bias':
                sum_rating += userdict[user][item][0] - mu - itemdict[item]['bias']
        userdict[user]['bias'] = sum_rating/(l + len(userdict[user]) - 1)
    if diff < 1e-10:
        break

In [21]:
# compute item-item similarity
item_sim = {}
sums = {}
for user in userdict:
    for ii in userdict[user]:
        if ii != 'bias':
            bui = mu + itemdict[ii]['bias'] + userdict[user]['bias']
            for ij in userdict[user]:
                if ij != 'bias' and ij != ii:
                    buj = mu + userdict[user]['bias'] + itemdict[ij]['bias']
                    if ii not in item_sim:
                        item_sim[ii] = {}
                    if ij not in item_sim[ii]:
                        item_sim[ii][ij] = 0
                    ratio = userdict[user][ii][1]/userdict[user][ij][1]
                    ratio = 1/ratio if ratio>1 else ratio
                    item_sim[ii][ij] += ratio * (userdict[user][ii][0] - bui) * (userdict[user][ij][0] - buj)
                    if str(ii)+' '+str(ij) not in sums:
                        sums[str(ii)+' '+str(ij)] = {ii: 0, ij: 0}
                    sums[str(ii)+' '+str(ij)][ii] += (userdict[user][ii][0] - bui)**2
                    sums[str(ii)+' '+str(ij)][ij] += (userdict[user][ij][0] - buj)**2

In [22]:
from math import sqrt
for ii in item_sim:
    for ij in item_sim[ii]:
        item_sim[ii][ij] /= sqrt(1e-16 + sums[str(ii)+' '+str(ij)][ii] * sums[str(ii)+' '+str(ij)][ij])
        item_sim[ii][ij] = item_sim[ii][ij]/2 + 0.5

In [23]:
testdata1 = []
testdata2 = []
for l in readJson('amazon_reviews_Electronics/helpful.json'):
    tmp1 = {}
    tmp2 = {}
    u,i,rating,outof,summary = l['reviewerID'],l['asin'],l['overall'],l['outOf'],l['summary']
    tmp1['user'] = u
    tmp1['rating'] = rating
    tmp1['outof'] = outof
    tmp1['summary'] = summary
    tmp2['item'] = i
    tmp2['rating'] = rating
    tmp2['outof'] = outof
    tmp2['summary'] = summary
    testdata1.append(tmp1)
    testdata2.append(tmp2)

In [24]:
test_df1 = pd.DataFrame(testdata1).set_index('user')
test_df1.head()

,outof,rating,summary
user,,,
3f8e4e55ed9642e497530b5782d4a34aed9703613d2779666d6a62833584d2a7,15,1.0,Very Disappointed
b587571ff6586ce380f855823e4396582ed68fe82ff42979c49905976e80434b,2,4.0,As expected
1e3497448fcff0a0155f43434b65d4773644b862fd0bb7e9292abe0fc41972f6,4,5.0,Bingo-Bango-Bongo
359fcfd1b6d053820af1cac1f2ffef7616d318d39e86c5b716f3629194343716,11,5.0,Good quality mount
50cbcf93ce240cd355dee473773c1f37fe4cc86a837ad2a351f7d0869b4a06e3,1,4.0,Excellent product for the money


In [27]:
user_test_sum = normalize(svd1.transform(vectorizer1.transform(test_df1.summary)))
test_df_user = pd.concat([test_df1.drop('summary', axis=1), pd.DataFrame(user_test_sum, columns=list(range(100)),index=test_df1.index)], axis=1)
test_df_user.head()

,outof,rating,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
user,,,,,,,,,,,,,,,,,,,,,
3f8e4e55ed9642e497530b5782d4a34aed9703613d2779666d6a62833584d2a7,15,1.0,0.007164,0.014875,-0.005629,0.046972,0.010879,0.048875,0.013479,0.047111,...,-0.109684,-0.009382,0.252628,0.086800,-0.223871,0.076634,0.115923,0.315842,0.335939,-0.123738
b587571ff6586ce380f855823e4396582ed68fe82ff42979c49905976e80434b,2,4.0,0.002881,0.004867,0.023951,0.008236,0.007026,0.004711,0.001177,0.005289,...,-0.022275,-0.013773,-0.000078,-0.001136,0.011978,0.007630,0.003500,0.016730,-0.000690,-0.004774
1e3497448fcff0a0155f43434b65d4773644b862fd0bb7e9292abe0fc41972f6,4,5.0,0.000886,0.001544,0.002680,0.000142,0.004691,0.012338,-0.001109,0.005639,...,-0.042553,-0.065543,0.509679,0.215225,-0.127053,0.053326,-0.135144,0.163611,0.099752,-0.008187
359fcfd1b6d053820af1cac1f2ffef7616d318d39e86c5b716f3629194343716,11,5.0,0.084960,0.718863,0.005470,-0.092552,0.017104,0.207043,-0.194632,0.372746,...,0.004372,-0.004093,-0.000465,-0.026977,0.053752,0.015654,0.022760,-0.012070,0.151364,-0.091746
50cbcf93ce240cd355dee473773c1f37fe4cc86a837ad2a351f7d0869b4a06e3,1,4.0,0.097972,0.106238,-0.146291,0.671889,0.068419,0.289946,-0.235682,-0.195594,...,-0.002748,-0.004000,-0.001919,0.006056,0.000839,-0.002045,-0.004372,-0.002860,-0.001813,-0.001891


In [29]:
test_df2 = pd.DataFrame(testdata2).set_index('item')
item_test_sum = normalize(svd2.transform(vectorizer2.transform(test_df2.summary)))

In [30]:
test_df_item = pd.concat([test_df2.drop('summary', axis=1), pd.DataFrame(item_test_sum, columns=list(range(100)),index=test_df2.index)], axis=1)
test_df_item.head()

,outof,rating,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
item,,,,,,,,,,,,,,,,,,,,,
8939d791e9dd035aa58da024ace69b20d651cea4adf6159d984872b44f663301,15,1.0,0.007507,0.017896,-0.003193,0.100234,-0.024218,0.028699,0.016330,-0.036963,...,-0.111828,-0.094163,-0.054262,0.273973,0.352492,-0.182633,0.053690,-0.128094,0.027930,0.217576
6116d31a297ceb0f8f69f6f71e924e47136fc70c6f5bf75c7af0363663760159,2,4.0,0.002978,0.004755,0.024419,0.007527,0.005561,0.000846,0.003476,-0.003985,...,-0.000091,0.004720,-0.003607,-0.000454,-0.029989,0.006874,-0.003027,0.039705,-0.013720,0.006274
13132c29b5bca0c7d8521c57179b9b73c605989314477d3a742578627eb84c42,4,5.0,0.029034,0.004579,0.602575,0.071820,0.027155,-0.029765,-0.009977,-0.006878,...,0.099187,-0.147054,0.044720,-0.086789,0.088881,-0.044605,0.031014,-0.077506,0.081685,0.036828
13132c29b5bca0c7d8521c57179b9b73c605989314477d3a742578627eb84c42,11,5.0,0.103610,0.727899,-0.017173,-0.047026,0.041561,0.053472,0.398106,-0.446249,...,0.049887,-0.012103,-0.000812,-0.077142,0.062400,-0.049587,-0.032114,-0.028814,-0.007543,0.068037
13132c29b5bca0c7d8521c57179b9b73c605989314477d3a742578627eb84c42,1,4.0,0.090911,0.089773,-0.065470,0.679656,-0.153870,-0.029621,0.285742,0.299673,...,-0.003103,-0.000569,-0.000682,-0.004856,-0.002998,-0.000292,0.002655,-0.003462,-0.001843,-0.002074


In [66]:
user_labels = kmeans1.predict(test_df_user)
item_labels = kmeans2.predict(test_df_item)

In [67]:
user_label_kmean = []
for user_label in user_labels:
    user_label_kmean.append(user_df1.ix[kmeans1.labels_==user_label].helpful.mean())

In [68]:
item_label_kmean = []
for item_label in item_labels:
    item_label_kmean.append(item_df2.ix[kmeans2.labels_==item_label].helpful.mean())

In [75]:
predictions = open("amazon_reviews_Electronics/predictions_Helpful_1.txt", 'w')
iternum = 0
lam = 0.5
for l in open("amazon_reviews_Electronics/pairs_Helpful.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue  
    user, item, outof = l.strip().split('-')
    if user not in userdict:
        user = 'new'
    if item not in itemdict:
        item = 'new'
    bu = userdict[user]['bias'] if user!='new' else 0
    bi = itemdict[item]['bias'] if item!='new' else 0
    bui = mu + bu + bi
    if user!='new' and item in item_sim:
        CF_predict = sum_sim = 0
        for other in item_sim[item]:
            buj = mu + userdict[user]['bias'] + itemdict[other]['bias']
            if other in userdict[user]:
                pop = userdict[user][other][1] if other in userdict[user] else mu_votes
                CF_predict += min(1, pop/5) * item_sim[item][other] * (userdict[user][other][0] - buj)
                sum_sim += item_sim[item][other]
        CF_predict = CF_predict/(lam + sum_sim) + bui
    else:
        CF_predict = bui
    if user == 'new' or item == 'new':
        predict = CF_predict/5 + 2*user_label_kmean[iternum]/5 + 2*item_label_kmean[iternum]/5
    else:
        predict = CF_predict/2 + user_label_kmean[iternum]/4 + item_label_kmean[iternum]/4
    predict *= int(outof)
    predictions.write(l.strip('\n') + ',' + str(predict) + '\n')
    iternum += 1
predictions.close()